# TRABAJO FINAL: PROCESAMIENTO DEL LENGUAJE NATURAL

## *Análisis de sentimientos: Las recetas de cocina y Facebook*
### Año 2017
### Integrantes
 * Franco, Laura Sol
 * Marrero Pizarro, Andrés Maximiliano
 * Timpani, Pamela Denise
 * Zamit, Ana Laura

### Objetivo del trabajo
Se implementará un clasificador que permita realizar un análisis de sentimientos a partir de los comentarios realizados por los usuarios de Facebook en publicaciones realizadas por páginas de cocina que contengan recetas.
### Abordaje del desarrollo e implementación del clasificador 
En los pasos que se irán detallando a continuación se indicará cómo se construye el clasificador, con qué bases de datos se cuenta, como se realiza la tokenización de las palabras, limpieza y acondicionamiento de los datos con los que se cuenta, desarrollo del clasificador del tipo suvervisado en Python y posterior implementación en un set testeo.
#### Importación de librerías
Las librerías de Python que se utilizarán para la labor son las siguientes:

In [1]:
import urllib3
urllib3.disable_warnings()
import json
import datetime
import csv
import time
import pandas as pd
import numpy as np
import re

#### Configuración de la API de Facebook
A continuación se destacan los parámetros utilizados para acceder a las páginas de Facebook de interés como *Upsoclsabores* o *Cucinareok* y tomar los comentarios de los usuarios.

1\. El usuario y la password utilizada junto con el token de acceso, los id de las páginas son:


In [2]:
app_id = "133254600775938"

app_secret = "c24db12ccb6a583d8e212600c1bed43e" # DO NOT SHARE WITH ANYONE!

access_token = app_id + "|" + app_secret

#page_id = 'nytimes'
#page_id = 'www.salesintacc.com.ar'
#page_id = 'upsoclsabores'
page_id = 'cucinareok'

2\. Acceso a Facebook: se genera un permiso para ingresar a la aplicación y en caso de error se reinicia la conexión.

In [3]:
def request_until_succeed(url):
    http = urllib3.PoolManager()
    req = http.request('GET', url)
    success = False
    while success is False:
        try:
            response = http.urlopen('GET',url)
            if response.status == 200:
                success = True
        except:
            time.sleep(5)
            
            print ("Error for URL %s: %s" % (url, datetime.datetime.now()))

    return req

In [41]:
def getFacebookPageFeedData(page_id, access_token, num_statuses):
    
    # construct the URL string
    base = "https://graph.facebook.com/v2.11" #cambié v2.4 a v2.11, funcionaba igual con v2.4
    node = "/" + page_id + "/feed"
    parameters = "/?fields=message,link,created_time,type,name,id,comments,from,attachment,shares&limit=%s&access_token=%s" % (num_statuses, access_token)
    url = base + node + parameters
    
    # Esto está cambiado respecto al ejemplo
    http = urllib3.PoolManager()
    req = http.request('GET', url)
    data = json.loads(request_until_succeed(url).data.decode('utf-8'))
    return data
    

In [42]:
#Estoy levantando 100 posts
salida_all = getFacebookPageFeedData(page_id, access_token, 100)

columns = ['post_from', 'post_id', 'post_name', 'post_type', 'post_message', 'post_link', 'post_shares', 'created_time']

df_posts = pd.DataFrame(columns=columns)

In [43]:
def scrapPosts(post):
    post_from = '' if 'from' not in post.keys() else post['from']['id']
    post_id = post['id']
    post_name = '' if 'name' not in post.keys() else post['name']
    post_type = '' if 'type' not in post.keys() else post['type']
    post_message = '' if 'message' not in post.keys() else post['message']
    post_link = '' if 'link' not in post.keys() else post['link']
    post_shares = '' if 'shares' not in post.keys() else post['shares']['count']
    created_time = datetime.datetime.strptime(post['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    created_time = created_time + datetime.timedelta(hours=-5) # EST
    created_time = created_time.strftime('%Y-%m-%d %H:%M:%S')    

    return(post_from, post_id, post_name, post_type, post_message, post_link, post_shares, created_time)


In [44]:
n_posts = len(salida_all['data'])

for i in range(n_posts):
    df = pd.DataFrame(list(scrapPosts(salida_all["data"][i])),index = ['post_from', 'post_id', 'post_name', 'post_type', 'post_message', 'post_link', 'post_shares', 'created_time']).transpose()
    df_posts = df_posts.append(df, ignore_index=True)    


In [64]:
df_posts.to_csv( 'df_posts.csv', encode = 'utf-8' )

In [282]:
def scrapComments(pos, com):

    post_id = salida_all["data"][pos]["id"]
    comm_id = salida_all["data"][pos]["comments"]["data"][com]["id"]
    comm_msg = salida_all["data"][pos]["comments"]["data"][com]["message"]
    comm_date = datetime.datetime.strptime(salida_all["data"][pos]["comments"]["data"][com]["created_time"],'%Y-%m-%dT%H:%M:%S+0000')
    comm_date = comm_date + datetime.timedelta(hours=-5) # EST
    comm_date = comm_date.strftime('%Y-%m-%d %H:%M:%S') # best time format for spreadsheet programs
    
    return(post_id, comm_id, comm_msg, comm_date)    


In [283]:
df_comms = pd.DataFrame(columns=['post_id','comm_id','comm_msg','comm_date'])

In [284]:
#n_posts = len(salida_all['data'])

for i in range(n_posts):
    if len(salida_all["data"][i]) <= 8:
        continue
    ncomm = len(salida_all["data"][i]["comments"]["data"])
    for c in range(ncomm):
        df = pd.DataFrame(list(scrapComments(i,c)), index=['post_id','comm_id','comm_msg','comm_date']).transpose()
        df_comms = df_comms.append(df, ignore_index=True)
        
        

In [285]:

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

df_comms['emojis'] = df_comms['comm_msg'].apply(lambda x: emoji_pattern.findall(x))

df_comms['comm_msg'] = df_comms['comm_msg'].apply(lambda x: emoji_pattern.sub(r'', x))

df_comms['emojis'] = df_comms['emojis'].apply(lambda x: ''.join(x))

In [286]:
df_comms['emoji_count'] = df_comms.emojis.str.len()

In [287]:
df_comms.to_csv( 'df_comms.csv', encoding = 'utf-8')

In [288]:
#reload_comms = pd.read_csv('df_comms.csv', encoding ='utf-8')
#reload_comms